In [1]:
import pandas as pd
import models
from recommenders import *

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump, load

In [3]:
movies_names = pd.read_table("ml-10M100K/movies.dat", sep="::", names=["id", "name", "tags"], index_col="id")
movies_names.drop(axis=1, columns=["tags"], inplace=True)
movies_names.head()

<ipython-input-3-8a3e7b26f09c>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_names = pd.read_table("ml-10M100K/movies.dat", sep="::", names=["id", "name", "tags"], index_col="id")


,name
id,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


In [4]:
movie_tags = pd.read_pickle("movielens10_tags_embedding/tags_all.pkl")
movie_tags.set_index("movieId", inplace=True)
movie_tags.head()

,tag
movieId,
1,Pixar Pixar Pixar animation Pixar animated fun...
2,For children game animals Joe Johnston Robin W...
3,Funniest Movies comedinha de velhinhos engraÃ§...
4,girl movie
5,steve martin pregnancy remake steve martin fam...


In [5]:
collab_movies = pd.read_pickle("movielens10_users_embedding/ratings_train.pkl").columns.to_numpy()

In [6]:
users_ratings: "pd.DataFrame" = pd.concat([pd.read_pickle("movielens10_users_embedding/ratings_train.pkl"),
                                           pd.read_pickle("movielens10_users_embedding/ratings_valid.pkl"),
                                           pd.read_pickle("movielens10_users_embedding/ratings_test.pkl")])
users_ratings.shape

(20685, 8023)

In [7]:
auto_rec_path = "movielens10_users_embedding/auto_encoder"
collab_recommender = RatingBasedRecommender(auto_rec_path, collab_movies)

In [91]:
user = users_ratings.sample(1)
recomms = collab_recommender.recommend(user.to_numpy(), 10, movies_names)
# print(movies_names.loc[recomms])

In [92]:
# recomms["names"] = movies_names.loc[recomms.index]

In [93]:
recomms

,rating,name
2762,4.472802,"Sixth Sense, The (1999)"
745,4.447361,Wallace & Gromit: A Close Shave (1995)
1198,4.354260,Raiders of the Lost Ark (Indiana Jones and the...
527,4.330620,Schindler's List (1993)
318,4.314923,"Shawshank Redemption, The (1994)"
3114,4.312569,Toy Story 2 (1999)
58559,4.292597,"Dark Knight, The (2008)"
6539,4.288939,Pirates of the Caribbean: The Curse of the Bla...
260,4.288165,Star Wars: Episode IV - A New Hope (a.k.a. Sta...
150,4.275661,Apollo 13 (1995)


In [94]:
user

movieId,1,2,3,4,5,6,7,8,9,10,...,63436,63853,63859,63876,63992,64030,64497,64614,64620,64957
62,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
movies_names.loc[user.where(user>0).dropna(axis=1).sort_values(by=user.index[0], ascending=False, axis=1).columns].head(10)

,name
movieId,
1356,Star Trek: First Contact (1996)
1073,Willy Wonka & the Chocolate Factory (1971)
832,Ransom (1996)
780,Independence Day (a.k.a. ID4) (1996)
708,"Truth About Cats & Dogs, The (1996)"
788,"Nutty Professor, The (1996)"
748,"Arrival, The (1996)"
736,Twister (1996)
719,Multiplicity (1996)


In [43]:
tag_recomm = TagBasedRecommender(autoencoder_path="movielens10_tags_embedding/auto_encoder", vectorizer_path="movielens10_tags_embedding/vectorizer.joblib", n_layers=12)

In [44]:
tag_recomm.calculate_similarity_matrix(movie_tags)

movieId     1         2         3         4         5         6         7      \
movieId                                                                         
1        1.000000  0.363732  0.413832  0.317689  0.327223  0.043090  0.354063   
2        0.363732  1.000000  0.411794  0.152473  0.602107  0.480541  0.536454   
3        0.413832  0.411794  1.000000  0.414638  0.418655  0.207821  0.463287   
4        0.317689  0.152473  0.414638  1.000000  0.169567  0.116914  0.145614   
5        0.327223  0.602107  0.418655  0.169567  1.000000  0.459447  0.693651   

movieId     8         9         10     ...     64942     64957     64959  \
movieId                                ...                                 
1        0.431334  0.389701  0.421854  ...  0.287272  0.107831  0.104651   
2        0.170911  0.593584  0.535112  ...  0.670030  0.565404  0.563311   
3        0.336980  0.464244  0.496210  ...  0.419102  0.249719  0.260396   
4        0.575577  0.233548  0.272443  ...  0.148808

In [41]:
d = pd.DataFrame(tag_recomm.similarity_matrix.loc[2])
d.columns=['sim']
d.head()
d.sort_values(by='sim', ascending=False).drop(index=2)

,sim
movieId,
3448,0.757875
500,0.755184
1246,0.740454
3105,0.728966
3283,0.725236
...,...
62439,0.025768
62792,0.025768
63033,0.025768


In [56]:
tag_recomm.recommend(movies_names, 1, 10)

,similarity,name
movieId,,
3114,0.776632,Toy Story 2 (1999)
2018,0.717052,Bambi (1942)
2355,0.709286,"Bug's Life, A (1998)"
2080,0.693575,Lady and the Tramp (1955)
8961,0.632617,"Incredibles, The (2004)"
3964,0.630069,"Adventures of Ichabod and Mr. Toad, The (1949)"
5533,0.617095,Children On Their Birthdays (2002)
6377,0.611607,Finding Nemo (2003)
3615,0.606088,Dinosaur (2000)


In [57]:
movies_names.loc[1]

name    Toy Story (1995)
Name: 1, dtype: object

In [70]:
tag_model = keras.models.load_model("movielens10_tags_embedding/auto_encoder")

In [73]:
tag_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_10 (Batc (None, 11352)             45408     
_________________________________________________________________
dropout_10 (Dropout)         (None, 11352)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 3000)              34059000  
_________________________________________________________________
batch_normalization_11 (Batc (None, 3000)              12000     
_________________________________________________________________
dropout_11 (Dropout)         (None, 3000)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1000)              3001000   
_________________________________________________________________
batch_normalization_12 (Batc (None, 1000)             

In [118]:
encoder = keras.models.Sequential()
encoder.add(tag_model.input)
for layer in tag_model.layers[:12]:
    encoder.add(layer)
encoder.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_10 (Batc (None, 11352)             45408     
_________________________________________________________________
dropout_10 (Dropout)         (None, 11352)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 3000)              34059000  
_________________________________________________________________
batch_normalization_11 (Batc (None, 3000)              12000     
_________________________________________________________________
dropout_11 (Dropout)         (None, 3000)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1000)              3001000   
_________________________________________________________________
batch_normalization_12 (Batc (None, 1000)             

In [103]:
# movie_tags.iloc[0]

tag    Pixar Pixar Pixar animation Pixar animated fun...
Name: 1, dtype: object

In [131]:
p = encoder.predict(vectorizer.transform(movie_tags["tag"]).todense())
p = pd.DataFrame(data=p, index=movie_tags.index)

In [126]:
movie_tags["tag"]

movie_tags

,tag
movieId,
1,Pixar Pixar Pixar animation Pixar animated fun...
2,For children game animals Joe Johnston Robin W...
3,Funniest Movies comedinha de velhinhos engraÃ§...
4,girl movie
5,steve martin pregnancy remake steve martin fam...
...,...
64993,anime anime drama Japan love
65006,doppleganger Nudity (Rear) Nudity (Topless - N...
65037,asperger's syndrome autism based on novel fake...


In [128]:
movie_tags.loc[1]

tag    Pixar Pixar Pixar animation Pixar animated fun...
Name: 1, dtype: object

In [132]:
p

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.000000,0.000000,0.000000,1.136376,0.000000,0.928714,1.527924,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.126168,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,3.092935,1.575060,3.963101,4.469917,0.000000,0.000000,0.545839,...,0.000000,0.000000,2.751279,0.000000,0.000000,0.719949,0.000000,0.928765,0.000000,0.000000
3,0.000000,2.113630,0.000000,0.559432,0.271731,2.274636,0.000000,1.354008,0.000000,2.300522,...,0.000000,0.000000,0.649049,0.695271,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,1.150429,3.093085,0.000000,0.000000,1.385180,0.000000,0.917971,0.288340,1.389271,...,0.000000,0.000000,0.000000,1.119013,2.710472,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,1.071377,0.627913,0.807075,0.000000,0.000000,0.000000,0.502802,...,0.000000,0.000000,2.960604,0.000000,0.000000,0.120545,3.225163,1.149298,0.686264,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64993,0.000000,0.000000,1.790812,0.410538,0.000000,1.657879,0.000000,0.967386,0.000000,0.049154,...,0.197314,0.000000,0.685837,1.282354,0.000000,1.912245,0.126529,1.069532,0.000000,0.000000
65006,0.000000,0.000000,0.000000,8.017126,0.000000,5.211590,11.065426,0.000000,0.000000,0.000000,...,0.000000,0.000000,3.246121,0.000000,0.000000,10.369312,6.046316,7.602953,0.000000,0.000000
65037,0.365008,0.000000,0.000000,0.830745,0.247003,2.953942,2.298719,0.000000,0.000000,1.865382,...,0.000000,0.897502,2.766974,0.000000,0.000000,1.003160,2.174779,2.200910,0.000000,0.000000


In [134]:
pd.DataFrame(p.loc[500], col)

,500
0,0.000000
1,0.000000
2,0.000000
3,2.686862
4,1.995593
...,...
195,1.218837
196,0.775563
197,0.765634
198,0.000000
